In [2]:
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models


## Load dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and normalize images
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float16') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float16') / 255

# One-hot encode labels (keep this as float32 for stability in the loss calculations)
train_labels = to_categorical(train_labels, dtype='float32')
test_labels = to_categorical(test_labels, dtype='float32')


model = models.Sequential()
model.add(layers.Conv2D(4, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((4, 4)))
model.add(layers.Conv2D(8, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10, dtype='float32'))  # Ensure that the final layer has float32 dtype

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the CNN model
model.fit(train_images, train_labels, epochs=30)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)


Epoch 1/30
1875/1875 [==============================] - 22s 11ms/step - loss: 0.6468 - accuracy: 0.8006
Epoch 2/30
1875/1875 [==============================] - 19s 10ms/step - loss: 0.2487 - accuracy: 0.9228
Epoch 3/30
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1966 - accuracy: 0.9395
Epoch 4/30
1875/1875 [==============================] - 21s 11ms/step - loss: 0.1705 - accuracy: 0.9469
Epoch 5/30
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1545 - accuracy: 0.9513
Epoch 6/30
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1442 - accuracy: 0.9548
Epoch 7/30
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1367 - accuracy: 0.9572
Epoch 8/30
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1307 - accuracy: 0.9589
Epoch 9/30
1875/1875 [==============================] - 21s 11ms/step - loss: 0.1260 - accuracy: 0.9606
Epoch 10/30
1875/1875 [==============================] - 19s 10m

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 4)         40        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 6, 6, 4)          0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 8)           296       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 2, 8)          0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 10)                3

In [4]:
layer_weights = []
layer_biases = []

for layer in model.layers:
    weights = layer.get_weights()

    # Check if the layer has weights and biases
    if len(weights) > 0:
        # Convert to float16
        w = weights[0].astype(np.float16)
        b = weights[1].astype(np.float16)

        layer_weights.append(w)
        layer_biases.append(b)
    else:
        # Append None for layers without weights/biases, e.g., MaxPooling layers
        layer_weights.append(None)
        layer_biases.append(None)

for idx, (weight, bias) in enumerate(zip(layer_weights, layer_biases)):
  if weight is not None:
      np.save(f'layer_{idx}_weights.npy', weight)
  if bias is not None:
      np.save(f'layer_{idx}_biases.npy', bias)

for idx, (weight, bias) in enumerate(zip(layer_weights, layer_biases)):
    if weight is not None:
        weight.tofile(f'layer_{idx}_weights.bin')
    if bias is not None:
        bias.tofile(f'layer_{idx}_biases.bin')

